In [6]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

TRAINING_DATA = "datasets/train.csv"
TRAIN_COUNT = 2500
TEST_COUNT = 1678
FULL_SET = 4178
IMG_SIZE = (48,48)
images = []
target = []

In [7]:
#Read CSV to pandas DataFrame
data = pd.read_csv(TRAINING_DATA)
Emotions = data['Emotion'].values
Pixels = data['Pixels'].values

#Formatting pixels to Image Matrix
for i in range(len(Pixels)):
    pixelsFormatted = np.fromstring(Pixels[i], dtype = int, sep = " ").reshape(IMG_SIZE)
    images.append(pixelsFormatted)
    target.append(int(Emotions[i]))    

In [8]:
X_train = np.array(images)

Y_train = np.array(target)

#Adding Color depth
X_train = X_train.reshape(X_train.shape[0],48,48,1)
#7 classes of Output
Y_train = np_utils.to_categorical(Y_train,7)
#Converting every Element to float
X_train = X_train.astype('float32')
#Normalize
X_train /= 255

In [10]:
model = Sequential()

model.add(Convolution2D(64, 3, data_format="channels_last", kernel_initializer="he_normal", 
                 input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Convolution2D(64, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.6))

model.add(Convolution2D(32, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Convolution2D(32, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Convolution2D(32, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.6))

model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.6))

model.add(Dense(7))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
batch_normalization_3 (Batch (None, 46, 46, 64)        256       
_________________________________________________________________
activation_3 (Activation)    (None, 46, 46, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
batch_normalization_4 (Batch (None, 44, 44, 64)        256       
_________________________________________________________________
activation_4 (Activation)    (None, 44, 44, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 64)        0         
__________

In [11]:
#Training Dataset
model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1)

#Save model to Disk
#model.save('blah.h5')


Epoch 1/10
4178/4178 [==============================] - 189s - loss: 2.0496 - acc: 0.3047   
Epoch 2/10
4178/4178 [==============================] - 187s - loss: 1.4929 - acc: 0.4818   
Epoch 3/10
4178/4178 [==============================] - 186s - loss: 1.2834 - acc: 0.5400   
Epoch 4/10
4178/4178 [==============================] - 185s - loss: 1.1641 - acc: 0.5864   
Epoch 5/10
4178/4178 [==============================] - 184s - loss: 1.0804 - acc: 0.6087   
Epoch 6/10
4178/4178 [==============================] - 187s - loss: 1.0193 - acc: 0.6266   
Epoch 7/10
4178/4178 [==============================] - 188s - loss: 0.9671 - acc: 0.6443   
Epoch 8/10
4178/4178 [==============================] - 198s - loss: 0.8960 - acc: 0.6778   
Epoch 9/10
4178/4178 [==============================] - 203s - loss: 0.8659 - acc: 0.6831   
Epoch 10/10
4178/4178 [==============================] - 199s - loss: 0.8207 - acc: 0.6989   


In [12]:
model.save('new.h5')